In [1]:
print('hi')

hi


In [2]:
ls

README.md         data/             pth/              utils.py
__pycache__/      epoch_fn.py       requirements.txt
augmix.py         main.py           test.ipynb
config.py         model/            tta_compare.py


In [3]:
cd data

/Users/aljam/projects/image_playground/data


In [4]:
cd cifar10

/Users/aljam/projects/image_playground/data/cifar10


In [7]:
!python prepare_data.py

Traceback (most recent call last):
  File "/Users/aljam/projects/image_playground/data/cifar10/prepare_data.py", line 9, in <module>
    from config import CFG
ModuleNotFoundError: No module named 'config'


In [13]:
from config import CFG

In [14]:
CFG.image_size

128

In [ ]:
from